In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [40]:
class LeNet:
    
    def __init__(self, dataset, ws=None, trainable=True, x_shape=[None,784], y_shape=[None,10]):
        '''
        
        trainable: if true, we will build a new LeNet model and train it; otherwise it means
                    that we already have a LeNet model and wanna to rebuild that old model,
                    this time, all weights in LeNet are fixed and non-trainable, instead, 
                    the inputs (xs) are changed into weights, we wanna to see what kind of these
                    inputs (xs) could trigger a perticular neuron to have its maximium output. 
        
        '''
        self.trainable=trainable
        self.x_shape=x_shape
        self.y_shape=y_shape
        
        if self.trainable:
            self.ws=[None for i in range(8)]
        else:
            self.ws=ws
        
        self.build_up_LeNet()
        
        if self.trainable:
            self.build_up_loss()      
            
    def build_up_LeNet(self):
        '''
        
        trainable: if true, build the model; if false, weights are non-trainable, and inputs
                    (xs) are trainable variables, and are initialized with ws.
        ws = [w_conv1, b_conv1, w_conv2, b_conv2, w_fc1, b_fc1, w_fc2, b_fc2]
        '''
        # build up the LeNet
            
        # 1st layer
        # Output: [-1,14,14,32]
        if self.trainable:
            self.x=tf.placeholder(tf.float32, self.x_shape)
        else:
            self.x=tf.Variable(tf.truncated_normal(shape=[1,784], stddev=0.1))
            
        self.y_=tf.placeholder(tf.float32, self.y_shape)
        self.x_img=tf.reshape(self.x, [-1,28,28,1])
            
        self.w_conv1=self.weight_variable([5,5,1,32], self.ws[0])
        self.b_conv1=self.bias_variable([32], self.ws[1])
            
        self.h_conv1=tf.nn.relu(self.conv2d(self.x_img, self.w_conv1)+self.b_conv1)
        self.h_pool1=self.max_pooling_2x2(self.h_conv1)
            
        # 2nd layer
        # Output: [-1,7,7,64]
        self.w_conv2=self.weight_variable([5,5,32,64], self.ws[2])
        self.b_conv2=self.bias_variable([64], self.ws[3])

        self.h_conv2=tf.nn.relu(self.conv2d(self.h_pool1, self.w_conv2)+self.b_conv2)
        self.h_pool2=self.max_pooling_2x2(self.h_conv2)
            
        # fully connected layer
        # Output: [-1,1024]
        self.w_fc1=self.weight_variable([7*7*64,1024], self.ws[4])
        self.b_fc1=self.bias_variable([1024], self.ws[5])

        self.h_pool2_flat=tf.reshape(self.h_pool2, [-1,7*7*64])
        self.h_fc1=tf.nn.relu(tf.matmul(self.h_pool2_flat, self.w_fc1)+self.b_fc1)
            
        # drop out
        # drop out rate = 1 - keep_prob
        # automatically rescall the neurons that are not chosen
        self.keep_prob=tf.placeholder(tf.float32)
        self.h_fc1_drop=tf.nn.dropout(self.h_fc1, self.keep_prob)
            
        # output layer
        self.w_fc2=self.weight_variable([1024,10], self.ws[6])
        self.b_fc2=self.bias_variable([10], self.ws[7])

        self.y_conv=tf.nn.softmax(tf.matmul(self.h_fc1_drop, self.w_fc2)+self.b_fc2)
    
    def build_up_loss(self):
        # loss function
        self.cross_entropy=tf.reduce_mean(tf.reduce_sum(-self.y_*tf.log(self.y_conv),
                                                                reduction_indices=[1]))
        self.correct_prediction=tf.equal(tf.arg_max(self.y_, 1), 
                                                 tf.arg_max(self.y_conv, 1))
        self.accuracy=tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
        # train
        self.train_step=tf.train.AdamOptimizer(1e-4).minimize(self.cross_entropy)
    
    def weight_variable(self, shape, w=None):
        if self.trainable:
            initial=tf.truncated_normal(shape, stddev=0.1)
        else:
            initial=w
        return tf.Variable(initial, trainable=self.trainable)
    
    def bias_variable(self, shape, b=None):
        if self.trainable:
            initial=tf.constant(0.1, shape=shape)
        else:
            initial=b
        return tf.Variable(initial, trainable=self.trainable)
    
    def conv2d(self, x, w):
        return tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME')
    
    def max_pooling_2x2(self, x):
        return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    def train(self, dataset):
        self.close()
        self.sess=tf.InteractiveSession()
        self.sess.run(tf.initialize_all_variables())
        
        for i in range(1000):
            batch=dataset.train.next_batch(100)
            if i%200 == 0:
                train_accuracy = self.accuracy.eval(
                    feed_dict={self.x:batch[0], self.y_: batch[1], self.keep_prob: 1.0})
                print("step %d, training accuracy %g"%(i, train_accuracy))
            self.train_step.run(feed_dict={self.x: batch[0], 
                                           self.y_: batch[1], self.keep_prob: 0.5})
    
    def train_feature(self, max_which_neuron):
        '''
        
        When the trainable is False, and we wanna max_which_neuron node in LeNet to be maximized,
        use this. Traing xs (inputs) without changing original weights, in this way we could find
        the feature for each layer in LeNet.
        '''
        if self.trainable:
            return
        
        self.close()
        self.sess=tf.InteractiveSession()
        self.sess.run(tf.initialize_all_variables())
        #
        # loss=tf.reduce_sum(max_which_neuron)
        # add L2 norm regularization
        #
        self.train_step=tf.train.GradientDescentOptimizer(0.2).minimize(\
            -max_which_neuron.eval().max()+0.5*tf.sqrt(tf.reduce_sum(tf.square(self.x))))
        
        for i in range(3000):
            self.train_step.run()
        
        return self.x.eval()
    
    def close(self):
        try:
            self.sess.close()
        except:
            pass

In [3]:
# load data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
# end

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# First, build the LeNet
m1=LeNet(dataset=mnist)
m1.train(mnist)

step 0, training accuracy 0.05
step 200, training accuracy 0.93
step 400, training accuracy 0.9
step 600, training accuracy 0.91
step 800, training accuracy 0.99


In [5]:
# record the model weights
ws=[m1.w_conv1.eval(), m1.b_conv1.eval(), m1.w_conv2.eval(), m1.b_conv2.eval()
    , m1.w_fc1.eval(), m1.b_fc1.eval(), m1.w_fc2.eval(), m1.b_fc2.eval()]

In [41]:
# maximize one activation
m2=LeNet(dataset=mnist, ws=ws, trainable=False)
a2=m2.train_feature(max_which_neuron=m2.h_pool2)

In [73]:
# max another neuron
a2=m2.train_feature(max_which_neuron=m2.h_pool2[0,0,0,1])

In [42]:
# visiualize the input which could maximize that activation
plt.imshow(m2.x_img.eval()[0,:10,:10,0], cmap='Greys_r')
plt.show()

In [12]:
# just visiualize the weights of the first layer
fig, axes=plt.subplots(4,8)
k=0
for i in axes.flat:
    i.imshow(m1.w_conv1.eval()[:,:,0,k], cmap='Greys_r')
    k=k+1 
plt.show()

In [39]:
m1.w_conv1.eval().max()

0.20003392